In [1]:
# Install essential packages
!pip install torch torchaudio librosa

# Clone the VITS repository
!git clone https://github.com/jaywalnut310/vits.git

# Change to the vits directory
%cd vits

# Install compatible version of scipy
!pip install scipy==1.8.1

# Modify requirements.txt to remove specific version constraints
!sed -i 's/Cython==0.29.21/Cython/g' requirements.txt
!sed -i 's/librosa==0.8.0/librosa/g' requirements.txt
!sed -i 's/matplotlib==3.3.1/matplotlib/g' requirements.txt
!sed -i 's/numpy==1.18.5/numpy/g' requirements.txt
!sed -i 's/scipy==1.5.2/scipy/g' requirements.txt
!sed -i 's/torch==1.6.0/torch/g' requirements.txt  # Allowing any version of torch

# Install the remaining requirements
!pip install -r requirements.txt

fatal: destination path 'vits' already exists and is not an empty directory.
/content/vits
  Using cached phonemizer-2.2.1-py3-none-any.whl.metadata (10 kB)
  Using cached tensorboard-2.3.0-py3-none-any.whl.metadata (2.0 kB)
ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3, 0.15.0
ERROR: Could not find a version that satisfies the requirement torchvision==0.7.0 (from versions: 0.12.0, 0.13.0, 0.13.1, 0.14.0, 0.14.1, 0.15.1, 0.15.2, 0.16.0, 0.16.1, 0.16.2, 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1)
ERROR: No matching distribution found for torchvision==0.7.0


In [2]:
# Create a directory for the data and extract the tar file
!mkdir -p /content/svarah_data
!tar -xf /content/svarah.tar -C /content/svarah_data

In [3]:
import os
import json
import librosa
import torchaudio
from sklearn.model_selection import train_test_split

# Define directories
audio_dir = "/content/svarah_data/svarah/audio"  # Update based on extracted structure
manifest_path = "/content/svarah_data/svarah/svarah_manifest.json"  # Path to the JSON manifest file

# Prepare dataset for training
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.wav')]
transcripts = []

# Load transcripts from the JSON manifest
with open(manifest_path, 'r') as f:
    for line in f:
        try:
            item = json.loads(line)
            # Assuming the JSON has 'audio_filepath' and 'text' keys
            audio_filepath = item['audio_filepath']
            transcript = item['text']
            transcripts.append((audio_filepath, transcript))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")

# Ensure the audio files are aligned with transcripts
audio_file_dict = {os.path.basename(f): f for f in audio_files}
transcripts_filtered = [(audio_file_dict[os.path.basename(audio_filepath)], transcript) for audio_filepath, transcript in transcripts if os.path.basename(audio_filepath) in audio_file_dict]

# Split into train, validation, test sets (80/10/10)
train_files, test_files, train_transcripts, test_transcripts = train_test_split([x[0] for x in transcripts_filtered], [x[1] for x in transcripts_filtered], test_size=0.2, random_state=42)
val_files, test_files, val_transcripts, test_transcripts = train_test_split(test_files, test_transcripts, test_size=0.5, random_state=42)

# Print the results
print(f'Training files: {len(train_files)}, Validation files: {len(val_files)}, Test files: {len(test_files)}')

Training files: 5324, Validation files: 666, Test files: 666


In [6]:
# Start training the model
!pip install unidecode
!pip install phonemizer
!pip install monotonic-align
!python train.py -c configs/base.json -m saved_model

# Training logs will show model progress

2024-11-05 16:08:51.888809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-05 16:08:51.928482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-05 16:08:51.939883: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-05 16:08:53.979671: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1039)
           2	LOAD_FAST(arg=0, lineno=1042)
           4	LOAD_CONST(arg=1, lineno=1042)
           6	BINARY_SUBSCR(arg=None, lineno=1042)
           8	LOAD_FAST(

In [8]:
# Install required packages from a requirements file
!pip install -r requirements.txt

# Additional required packages for audio processing and phoneme conversion
!pip install librosa torchaudio unidecode phonemizer monotonic-align

from IPython.display import Audio
import librosa

# Define a sample rate to use for audio playback
SAMPLE_RATE = 22050  # Set this to your desired sample rate

# Load a sample output for evaluation from the test files
sample_audio, _ = librosa.load(test_files[0], sr=SAMPLE_RATE)

# Play the audio sample at the specified sample rate
Audio(sample_audio, rate=SAMPLE_RATE)

  Using cached Cython-0.29.21-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached librosa-0.8.0.tar.gz (183 kB)
  Preparing metadata (setup.py) ... done
  Using cached matplotlib-3.3.1.tar.gz (38.8 MB)
  Preparing metadata (setup.py) ... done
  Using cached numpy-1.18.5.zip (5.4 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import websockets
import soundfile as sf
import asyncio
import nest_asyncio
import os

# Allow nested event loops (especially useful in environments like Google Colab)
nest_asyncio.apply()

async def audio_handler(websocket, path):
    async for message in websocket:
        # Assume `message` is WAV data from the client
        # Save the WAV data as a temporary file for conversion
        with open("temp.wav", "wb") as wav_file:
            wav_file.write(message)

        # Read the WAV file and convert it to FLAC
        data, samplerate = sf.read("temp.wav")
        sf.write("output.flac", data, samplerate, format='FLAC')

        # Send the FLAC data back to the client
        with open("output.flac", "rb") as flac_file:
            await websocket.send(flac_file.read())

async def start_server():
    port = 8765  # Initial port to try
    server_started = False

    while not server_started:
        try:
            start_server = await websockets.serve(audio_handler, "0.0.0.0", port)
            server_started = True
            print(f"Server started on port {port}")
        except OSError as e:
            if e.errno == 98:  # Port is in use
                print(f"Port {port} is in use. Trying next port...")
                port += 1  # Increment the port and try again
            else:
                raise e

    await start_server.wait_closed()

# Run the server using the existing event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(start_server())

Port 8765 is in use. Trying next port...
Port 8766 is in use. Trying next port...
Server started on port 8767
